<center>
<img src="../../img/ml_theme.png">
# Майнор "Интеллектуальный анализ данных" 
# Курс "Введение в анализ данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель ФКН НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="http://www.microsoft.com/en-us/openness/default.aspx#Ms-RL">Ms-RL</a>. Можно использовать в любых целях, но с обязательным упоминанием автора курса и аффилиации.

# Домашнее задание № 6. Xgboost в соревновании по кредитному скорингу

[Соревнование](https://inclass.kaggle.com/c/beeline-credit-scoring-competition)

Здесь мы будет прогнозировать, что человек просрочит выплаты по кредиту на 3 месяца и более (целевой признак - Delinquent90).

Признаки клиентов банка:

- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xgboost as xgb
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Считаем данные.**

In [2]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [3]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [4]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Дополнительно: исследование признаков, построение новых.**

**Посчитайте число пропусков в каждом признаке для обучающей и тестовой выборок.**

In [ ]:
for col in train_df.columns:
    # Ваш код здесь

In [ ]:
for col in test_df.columns:
    # Ваш код здесь

**Замените пропуски медианными значениями. Дополнительно: заменить пропуски как-то по-другому (например, прогнозировать значения для пропусков).**

In [ ]:
# Ваш код здесь

**Сформируйте объект dtrain типа xgb.DMatrix, которым будет представляться обучающая выборка.**

In [ ]:
dtrain = xgb.DMatrix # Ваш код здесь

**Зафисксируем параметры Xgboost. Дополнительно: использовать GridSearch, RandomizedGridSearch или методы библиотеки [Hyperopt](https://github.com/hyperopt/hyperopt) для оптимизации параметров.**

In [ ]:
param = {}
param['objective']           = "binary:logistic"
param['eval_metric']         = "auc" 
param['booster']             = "gbtree"
param['eta']                 = 0.01
param['max_depth']           = 6
param['subsample']           = 0.5
param['colsample_bytree']    = 0.5 
param['min_child_weight']    = 0

**Проведите 3-кратную кросс-валидацию с 500 итерациями Xgboost с помощью xgb.cv.**

In [ ]:
cv_results = xgb.cv # Ваш код здесь

**Получился DataFrame с результатами на обучающей и валидационной выборках на каждой итерации. Опеределите лучшее число итераций бустинга, при котором средний AUC на валидационной выборке (test-auc-mean) максимален.**

In [ ]:
best_num_rounds = # Ваш код здесь

**Обучите найденное "лучшее" число итераций Xgboost на всей обучающей выбоке.**

In [ ]:
best_xgb = xgb.train # Ваш код здесь

**Сделайте прогноз для тестовой выборки, предварительно создав из нее объект типа xgb.DMatrix.**

In [ ]:
dtest = xgb.DMatrix # Ваш код здесь
xgb_pred = best_xgb.predict(dtest)

**Запишите прогноз в файл и сделайте посылку в соревнование Kaggle.**

In [ ]:
def write_to_submission_file(predicted_probs, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_probs,
                                index = np.arange(75000, 
                                                  predicted_probs.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file(xgb_pred, '../../output/credit_scoring_first_xgb_pred.csv')